# Predicting Immunotherapy Response based on RNA-Seq

### Importing Data and gene pathways
TCGA Data

In [1]:
import pandas as pd
import numpy as np

tpm = pd.read_csv("data/tcga_sample/expression.tsv", sep="\t")
survival = pd.read_csv("data/tcga_sample/survival.tsv", sep="\t", skiprows=1, header=None)
meta = pd.read_csv("data/tcga_sample/metadata.tsv", sep="\t", skiprows=1, header=None)
cytokines = pd.read_csv("data/genes.cytokine_immune.txt", skiprows=2, header=None)

### Data Processing

Get the TPM values for cytokines pathway

In [2]:
# Only use cytokine expression
tpm = tpm.reindex(cytokines.iloc[:,0].unique(), axis='columns')
tpm = tpm.dropna(axis=1)

# perform quantile normalization
# https://stackoverflow.com/questions/37935920/quantile-normalization-on-pandas-dataframe
tpm /= np.max(np.abs(tpm),axis=0) # scale between [0,1]
rank_mean = tpm.stack().groupby(tpm.rank(method='first').stack().astype(int)).mean()
tpm = tpm.rank(method='min').stack().astype(int).map(rank_mean).unstack()

# convert pandas df to np array
tpm = tpm.values
survival = survival.iloc[:,1:3].values

Split data into training and testing sets

In [3]:
# split the data into a training set and a validation set
VALIDATION_SPLIT = 0.8

# indices = np.arange(tpm.shape[0])
# np.random.shuffle(indices)
# # tpm = tpm[indices]
# labels = surv_time[indices]
num_validation_samples = int(VALIDATION_SPLIT * tpm.shape[0])

x_train = tpm[:num_validation_samples]
y_train = survival[:num_validation_samples]
x_val = tpm[num_validation_samples:]
y_val = survival[num_validation_samples:]

### Survival Neural Network

Defining the loss function

In [4]:
import tensorflow as tf
import keras.backend as K

def negative_log_partial_likelihood_loss(regularization):
    #Wrapper function for the negative logg partial likelihood loss function
    
    def loss(y_true, risk):
        return negative_log_partial_likelihood(y_true, risk, regularization)
    return loss

def negative_log_partial_likelihood(censor, risk, regularization):
    """Return the negative log-partial likelihood of the prediction
    y_true contains the survival time
    risk is the risk output from the neural network
    censor is the vector of inputs that are censored
    regularization is the regularization constant (not used currently)
    
    Uses the Keras backend to perform calculations
    
    Sorts the surv_time by sorted reverse time
    """
    
    # calculate negative log likelihood from estimated risk
    K.print_tensor(censor)
    K.print_tensor(risk)
    hazard_ratio = K.exp(risk)
    log_risk = K.log(tf.cumsum(hazard_ratio)) # cumsum on sorted surv time accounts for concordance
    uncensored_likelihood = risk - log_risk
    censored_likelihood = uncensored_likelihood * censor
    num_observed_events = K.sum(censor)
    neg_likelihood = - K.sum(censored_likelihood) / tf.cast(num_observed_events, tf.float32)
    return neg_likelihood

Using TensorFlow backend.


In [5]:
# np.random.seed(123)
# negative_log_partial_likelihood(y_train, np.random.rand(y_train.shape[0], 2), 0).eval(session=K.get_session())

In [6]:
def batch_iter(data, labels, batch_size, shuffle=True, isValidationSet=False):
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1

    # Sorts the batches by survival time
    def data_generator():
        data_size = len(data)
        while True:
            # Sample from the dataset for each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                X, y = shuffled_data[start_index: end_index], shuffled_labels[start_index: end_index]
                
                # Sort X and y by survival time in each batch
                idx = np.argsort(abs(y[:,0]))[::-1]
                X = X[idx, :]
                y = y[idx, 1].reshape(-1,1) # sort by survival time and take censored data

                # reshape for matmul
                y = y.reshape(-1,1) #reshape to [n, 1] for matmul
                
                yield X, y

    return num_batches_per_epoch, data_generator()

train_steps, train_batches = batch_iter(x_train, y_train, 5)
next(train_batches)

(array([[0.00770439, 0.00770439, 0.09777909, 0.08354589, 0.14483557,
         0.03750752, 0.06533576, 0.09507118, 0.10876231, 0.13890046,
         0.15091908, 0.12206171, 0.48187445, 0.00770439, 0.00770439,
         0.00770439, 0.16835241, 0.11855542, 0.04477338, 0.04356357,
         0.08782074, 0.08175386, 0.16017084, 0.08354589, 0.36083847,
         0.10876231, 0.08577805, 0.05328115, 0.05555696, 0.05104396,
         0.07452724],
        [0.00770439, 0.10876231, 0.00770439, 0.05433715, 0.01549208,
         0.06885928, 0.00770439, 0.03144983, 0.0427311 , 0.00770439,
         0.05784024, 0.05937713, 0.00770439, 0.00770439, 0.00770439,
         0.00770439, 0.03144983, 0.02375978, 0.06885928, 0.05104396,
         0.06533576, 0.09256924, 0.05328115, 0.04726486, 0.02430823,
         0.03998144, 0.06391589, 0.0357079 , 0.02287782, 0.04186489,
         0.05937713],
        [0.00770439, 0.00770439, 0.04644952, 0.0125152 , 0.06056726,
         0.06533576, 0.09777909, 0.04186489, 0.03750752, 0.

In [29]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import Sequential
from keras.optimizers import Adam

BATCH_SIZE = 30

model = Sequential()
model.add(Dense(128, input_dim=x_train.shape[1], name="input"))
model.add(Dense(64, activation='relu', name="dense_1"))
model.add(Dropout(0.25, name="dropout_1"))
model.add(Dense(64, activation='relu', name="dense_2"))
model.add(Dense(1, activation='linear', name="output"))

opt = Adam(lr=0.001)

model_loss = negative_log_partial_likelihood_loss(0)

model.compile(optimizer=opt, loss=model_loss, metrics=['accuracy']) # Accuracy is meaningless in this case, only look at loss

train_steps, train_batches = batch_iter(x_train, y_train, BATCH_SIZE)
valid_steps, valid_batches = batch_iter(x_val, y_val, BATCH_SIZE)
history = model.fit_generator(train_batches, train_steps, epochs=20, validation_data=valid_batches, validation_steps=valid_steps)

Epoch 1/20
3/3 [==============================] - 1s 281ms/step - loss: 2.4061 - acc: 0.3031 - val_loss: 1.9918 - val_acc: 0.1500
Epoch 2/20
3/3 [==============================] - 0s 7ms/step - loss: 2.3646 - acc: 0.3281 - val_loss: 1.9887 - val_acc: 0.1500
Epoch 3/20
3/3 [==============================] - 0s 9ms/step - loss: 2.4264 - acc: 0.3031 - val_loss: 1.9831 - val_acc: 0.1500
Epoch 4/20
3/3 [==============================] - 0s 9ms/step - loss: 2.4286 - acc: 0.3083 - val_loss: 1.9793 - val_acc: 0.1500
Epoch 5/20
3/3 [==============================] - 0s 9ms/step - loss: 2.3701 - acc: 0.2875 - val_loss: 1.9774 - val_acc: 0.1500
Epoch 6/20
3/3 [==============================] - 0s 10ms/step - loss: 2.3496 - acc: 0.2729 - val_loss: 1.9774 - val_acc: 0.1500
Epoch 7/20
3/3 [==============================] - 0s 9ms/step - loss: 2.3580 - acc: 0.2646 - val_loss: 1.9772 - val_acc: 0.1500
Epoch 8/20
3/3 [==============================] - 0s 9ms/step - loss: 2.3559 - acc: 0.2573 - val_loss

In [30]:
predictions = model.predict(x_val)
predictions

array([[ 0.13205379],
       [-0.41463694],
       [-0.55275315],
       [-0.04706205],
       [-0.5039119 ],
       [ 0.08584359],
       [-0.10762785],
       [ 0.04295709],
       [ 0.12689504],
       [-1.5880321 ],
       [ 0.4125125 ],
       [-0.16764246],
       [-0.2638064 ],
       [-2.3235643 ],
       [-0.4737324 ],
       [-0.37796897],
       [-0.3604821 ],
       [-0.02432366],
       [-1.087952  ],
       [ 0.4487742 ]], dtype=float32)

In [31]:
from lifelines.utils import concordance_index

predictions_time = np.exp(predictions)
concordance_index(y_val[:,0], predictions_time, y_val[:,1])

0.6

In [32]:
predictions_time

array([[1.1411697 ],
       [0.66058004],
       [0.5753636 ],
       [0.9540282 ],
       [0.6041626 ],
       [1.0896358 ],
       [0.89796174],
       [1.0438931 ],
       [1.1352979 ],
       [0.2043273 ],
       [1.5106084 ],
       [0.84565616],
       [0.76812226],
       [0.09792393],
       [0.62267387],
       [0.6852518 ],
       [0.6973401 ],
       [0.9759698 ],
       [0.33690578],
       [1.5663909 ]], dtype=float32)

In [28]:
y_val[:,0]

array([ 158,  247,   56,  284,   77,   65,  955,  273,  214,  271, 3617,
       3056,  218,  178,  202,   55,  322,  237, 3767, 2292])